In [17]:
import pandas as pd
import numpy as np

import os
import pickle
import json
import re
import sys
from IPython.display import display

import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import Dataset, DataLoader

import importlib

import preprocessing_utils

importlib.reload(preprocessing_utils)

<module 'preprocessing_utils' from '/Users/izzy/quantnet/quantnet_encodings/Quantlet/Create_description/preprocessing_utils.py'>

In [15]:
parsed_Qs_file = '../../data/preprocessed/Quantlet/Parsed_Qs_12032023.pkl'
dataset = preprocessing_utils.QuantletDataset(parsed_Qs_file)

In [35]:
code_script_path = os.path.join(dataset.parsed_Qs_file.folder_name.iloc[30], dataset.parsed_Qs_file.script_name.iloc[30])

In [23]:
code_script_path

'../../data/QuantLet/UYCTR/LRtest.R'

In [32]:
g = dataset.parsed_Qs_file

In [34]:
g.query('type_script == "m"')

,folder_name,metainfo_file,type_script,script_name,script_name_no_ext
30,../../data/QuantLet/Stable/Stable_LogDensity_F...,{'Name of Quantlet': 'Stable_LogDensity_Financ...,m,Stable_LogDensity_Financials.m,Stable_LogDensity_Financials
31,../../data/QuantLet/Stable/Stable_Rescaling,"{'Name of Quantlet': 'Stable_Rescaling', 'Publ...",m,Stable_Rescaling.m,Stable_Rescaling
59,../../data/QuantLet/ssmmatlab/SSM_agtrimanssbs_d,"{'Name of QuantLet': 'SSM_agrimanssbs_d', 'Pub...",m,SSM_agrimanssbs_d.m,SSM_agrimanssbs_d
60,../../data/QuantLet/ssmmatlab/SSM_varma2echelon_d,"{'Name of QuantLet': 'SSM_varma2echelon_d', 'P...",m,varmax2echelon_d.m,varmax2echelon_d
61,../../data/QuantLet/ssmmatlab/SSM_viviusa_d,"{'Name of QuantLet': 'SSM_viviusa_d', 'Publish...",m,SSM_viviusa_d.m,SSM_viviusa_d
...,...,...,...,...,...
8600,../../data/QuantLet/SurrogateModel/SurrogateMo...,"{'Name of Quantlet': 'SRM_fig4.2', 'Published ...",m,main.m,main
8601,../../data/QuantLet/SurrogateModel/SurrogateMo...,"{'Name of Quantlet': 'SRM_fig4.2', 'Published ...",m,Plots.m,Plots
8602,../../data/QuantLet/SurrogateModel/SurrogateMo...,"{'Name of Quantlet': 'SRM_fig4.2', 'Published ...",m,Problem.m,Problem
8603,../../data/QuantLet/SurrogateModel/SurrogateMo...,"{'Name of Quantlet': 'SRM_fig4.2', 'Published ...",m,main.m,main


In [36]:

importlib.reload(preprocessing_utils)
print(preprocessing_utils.load_script(code_script_path))

%% Stable_LogDensity_Financials
    % Compares parametric densities of Normal, Stable and Cauchy with
    % empirical financial market returns on log-scale to emphasize
    % non-normal behavior.

%% Global Commands

% ver % verify the version of Matlab
clear;clc; % clear

% set global commands for font size and line width
size_font=12;
size_line=2;
set(0,'DefaultAxesFontSize',size_font,'DefaultTextFontSize',size_font);
set(0,'defaultlinelinewidth',size_line)

%% Data and dimensions

load data.mat
ret=price2ret(prices);
[A,B]=size(ret);

%% Plotting

scale=0.0001;
line_width=3;

h=figure();

% closed form pdfs for normal
x=-10:0.01:10;
pd_gauss = makedist('Normal',0,1);
pdf_gaussian = pdf(pd_gauss,x);
semilogy(x,pdf_gaussian+scale,'Color',[0 102 204]./255,'linewidth',line_width); hold on;

% closed form pdfs for cauchy
pdf_cauchy = tpdf(x,1);
semilogy(x,pdf_cauchy+scale,'Color',[204 0 0]./255,'linewidth',line_width)

% approx for stable
%ret_choice=ret(:,1);
pd_stable = makedist('Stabl